# Decomposing Time Series

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.datasets import co2
from statsmodels.tsa.seasonal import STL

In [ ]:
data = co2.load().data

In [ ]:
data

In [ ]:
data.plot()

Looking at the above, there are clearly some missing values.

In [ ]:
data.isna().sum()

In [ ]:
data = data.resample('ME').mean().ffill()

In [ ]:
data.plot()

In [ ]:
data.isna().sum()

There are several methods in the statsmodels library for decomposing time series.
* Examples: https://www.statsmodels.org/devel/tsa.html#time-series-filters

In [ ]:
data

In [ ]:
STL(data['co2'])

In [ ]:
res = STL(data['co2']).fit()
res.plot()
plt.show()

In [ ]:
data

In [ ]:
data[1:24].plot()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot

print('Autocorrelation')
acf_plot = plot_acf(data['co2'], lags=24, title='Autocorrelation')
pacf_plot = plot_pacf(data['co2'], lags=24, title='Partial Autocorrelation')

If you're looking at a model, a plot of the autocorrelation of the residuals will tell you whether some information or variables may be missing in your model.

The residuals for the CO2 data do not have a significant autocorrelation.

In [ ]:
acf_plot = plot_acf(res.resid, lags=24, title='Autocorrelation')

But if we look at one of our earlier examples on fitting data with polynomials, we can see how the autocorrelation of the residuals indicates that one order of polynomial is not as good for modeling as another.

In [ ]:
import numpy as np
import seaborn as sns

# generate 100 points from a normal 
# distribution that has mean = 0 and std dev = 3.5
np.random.seed(42)
noise = np.random.normal(0,3.5,100)

# make the data arrays (y = x^2 + noise)
x = np.linspace(0,10,100)
y = x**2 + noise
z1 = np.polyfit(x,y,1)
y1 = z1[1] + z1[0]*x
resid1 = y - y1
z2 = np.polyfit(x,y,2)
y2 = z2[2] + z2[1]*x + z2[0]*x**2
resid2 = y - y2

# plot them with order 1 and 2 fits
fig,ax = plt.subplots(2,2)

ax[0,0].plot(x,y,'ko')
ax[0,0].plot(x,y1,'b')
ax[0,1].plot(x,resid1,'ko')
ax[1,0].plot(x,y,'ko')
ax[1,0].plot(x,y2,'b')
ax[1,1].plot(x,resid2,'ko')

In [ ]:
acf_plot = plot_acf(resid1, lags=100-1, title='Autocorrelation')

In [ ]:
acf_plot = plot_acf(resid2, lags=100-1, title='Autocorrelation')

Back to the CO2 data.

We can look at long-term behavior for each of our shorter-term measures.

In [ ]:
m_plot = month_plot(data['co2'])

What does this tell us?

How about quarterly data?

In [ ]:
q_plot = quarter_plot(data['co2'])

We need to resample our data.  (This is a common time-series operation).

In [ ]:
data_quarterly = data.resample('QE').mean()

In [ ]:
data_quarterly

In [ ]:
q_plot = quarter_plot(data_quarterly['co2'])

## Seasonal Decomposition with moving average

In [ ]:
data

In [ ]:
a = data.reset_index()

In [ ]:
a.columns = ['Date','fit']

In [ ]:
a

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

ss_decomposition = seasonal_decompose(x=a['fit'], model='additive', period=12)
estimated_trend = ss_decomposition.trend
estimated_seasonal = ss_decomposition.seasonal
estimated_residual = ss_decomposition.resid

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False)
fig.set_figheight(10)
fig.set_figwidth(15)

axes[0].plot(a['Date'], a['fit'], label='Original')
axes[0].legend(loc='upper left');

axes[1].plot(a['Date'], estimated_trend, label='Trend')
axes[1].legend(loc='upper left');

axes[2].plot(a['Date'], estimated_seasonal, label='Seasonality')
axes[2].legend(loc='upper left');

axes[3].plot(a['Date'], estimated_residual, label='Residuals')
axes[3].legend(loc='upper left');

In [ ]:
plt.plot(a['Date'], estimated_seasonal, label='Seasonality')

In [ ]:
estimated_seasonal

In [ ]:
plt.plot(a.loc[515-5*12:525, 'Date'], estimated_seasonal.loc[515-5*12:525], label='Seasonality')